# Read SRX Calibartion file (xml version) and PSF.db file to save PSF tiff


 Code a mix of code from Bruker, Sarah Aufmkolk and Laura Breimann

This code was provided by Winfried Wiegraebe (Bruker Corporation) and adapted and commented by Sarah Aufmkolk to be distributed in the Wu lab. Winfried Wiegraebe produced the functions to read in the raw PSF data.

Laura changed the code to read in xml files (instead of json) for older images and added a part to save the image as a tiff stack. 

This script allows to plot the PSF that was acquired during the second step of the system calibration for biplane-setups as it is necessary on Bruker VXL and Vutara 352 microscopes with 3D SMLM capabilities.
The general outline for the folder structure used by SRX is described in the this manual chapter:
https://guide.vutara.bruker.com/m/11201/l/596854-how-vutara-stores-data

If you don't have access to the manual contact a Bruker representative.


Please acknowledge that depending if you are a Mac or Windows user, your syntax for your path will be different. Adapt accordingly. 
The following line allows you to ask for the current directory and therefore hints to the syntax. 

os.getcwd()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import xml.etree.ElementTree as ET
import tifffile as tf

In [ ]:
# Path to calibration folder. Please identify the path where the Calibration file is located. 
calibration_path = "/Users/laurabreimann/Desktop/Guys_data/PSFs/2017-08-03-19-34-25/Location-02/Calibration/"

## Meta data of the calibration

Recent versions of SRX replaced .xml with .json files to save settings. The file calibration.json contains information about the PSF used for fitting localizations and the dimensions of the raw data. This information will be read and assigned to the variable "calibration_data".

Here we use the older version with an .xml file:

In [ ]:
# Load the xml file
tree = ET.parse(calibration_path +'calibration.xml')
root = tree.getroot()

In [ ]:
# Get tha data associated with the PSF (other data could also be loaded here)
psf = root.find('PSF')

In [ ]:
# Access data from the XML elements
biplane_module = root.find('BiplaneModule')
biplane_psf_red = biplane_module.find('.//BiplanePSF[@Name="Red"]')

objective = root.find('Objective')
general = root.find('General')
psf = root.find('PSF')

# Extract specific attributes
red_fwhm_warning = biplane_psf_red.get('FWHMWarning0')

magnification = objective.get('Magnification')
na = objective.get('NA')
dichroic = general.get('Dichroic')
pixel_size_x = psf.get('PixelSizeX')
dim_y = psf.get('DimY')
dim_z = psf.get('DimZ')

# Print extracted data
print("Red FWHM Warning:", red_fwhm_warning)
print("Magnification:", magnification)
print("NA:", na)
print("Dichroic:", dichroic)
print("Pixel Size X:", pixel_size_x)
print("Dim Y:", dim_y)
print("Dim Z:", dim_z)

The dimensions of the PSF calibration, which is an average of all chosen PSFs in the second step of the Super-resolution biplane calibration, is read out in the following. This allows us to display the center on the PSF representation later.

In [ ]:
# Save the dimensions as variables
dim_x_psf = psf.get('DimX')
dim_y_psf = psf.get('DimY')
dim_z_psf = psf.get('DimZ')

In [ ]:
# Convert the dimension strings to integers
dim_z_psf = int(dim_z_psf)
dim_x_psf = int(dim_x_psf)
dim_y_psf = int(dim_y_psf)

## Technical information of the PSF file in dbl format: 
The pixel data for the PSF is stored in .dbl files in float32 format with an offset of 128. We read the data directly into a numpy array.
There is a 64 bit double float variant that we use for the "theoretical" psf that is stored within SRX, but all of the experimental psfs are 32 bit floats.
The header (128 bytes total) contains:
x dimension (4 byte int)
y dimension (4 byte int)
z dimension (4 byte int)
zstep (4 byte float)
x pixel size (4 byte float)
y pixel size (4 byte float)
unused (104 bytes)

These values should match up with those in the json file.

The naming convention for the PSF "PSF_{wavelength_used}{source_biplane}.dbl"

For Alexa Fluor 647, we use the 640nm laser, which has the SRX name: Red. Therefore the PSF would be stored in the file "PSF_Red0.dbl".


In [ ]:
filename = 'PSF_Red0.dbl'
file_path = os.path.join(calibration_path, filename)

# Read the header information
header_size = 128
with open(file_path, 'rb') as file:
    x_dim, y_dim, z_dim, z_step, x_pixel_size, y_pixel_size = np.fromfile(file, dtype=np.float32, count=6)

# Calculate the total number of elements to read
total_elements = dim_z_psf * dim_x_psf * dim_y_psf

# Read binary data and reshape
with open(file_path, 'rb') as file:
    file.seek(header_size)  # Skip the header
    psf_data = np.fromfile(file, dtype=np.float32, count=total_elements)

psf = psf_data.reshape(dim_z_psf, dim_x_psf, dim_y_psf)

In [ ]:
#The following three plots show the PSF in xy, xz and yz. The PSF is saved as a 3D matrix, if plotted the user has to chose a plane to be depicted. 
#Per default the center of the plane is displayed.

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 20))
ax[0].imshow(psf[dim_z_psf // 2, :, :])
ax[1].imshow(psf[:, dim_x_psf // 2, :])
ax[2].imshow(psf[:, :, dim_y_psf // 2])

In [ ]:
#If you want to see a specific slice of your PSF you can change the "dim_z_psf // 2" arguement to an absolute number.
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 20))
ax[0].imshow(psf[16, :, :])
ax[1].imshow(psf[:, 10, :])
ax[2].imshow(psf[:, :, 10]);

In [ ]:
output_directory = "/Users/laurabreimann/Desktop/Guys_data/PSFs/2017-08-03-19-34-25/Location-02/Calibration/"

In [ ]:
# Save the PSF data as a TIFF file
output_tiff_path = os.path.join(output_directory, 'psf_output.tif')  # Change 'output_directory' and file extension as needed
tf.imwrite(output_tiff_path, psf)